---

# Introduction

This is the naivest implementation considering that most items in the passages follow specific positional patterns.

I am using this notebook to demonstrate that such an approach can be used as a final filter for submitting the predictions from the main DL models.

This notebook only uses RandomForest. No deep learning Or complicated processing. Enjoy !

> In this version I changed the **RandomForest** to **GradientBoosting** model for prediction

---


In [ ]:
import pandas as pd

import numpy as np
import os
import re

from tqdm.auto import tqdm

import pandas as pd
import os
import re
from tqdm.auto import tqdm
import numpy as np

In [ ]:
ISLOCAL = False
V2 = True #Version 2 where we try to work on discourse_type number field

IS_CORRECTED_TRAIN = True
TRAIN_CHAR_LEVEL = True

if V2:
    PREDICTOR_DISCOURSE = "discourse_type_num"
else:
    PREDICTOR_DISCOURSE = "discourse_type"
if ISLOCAL:
    if IS_CORRECTED_TRAIN:
        train_csv = "./corrected_train.csv"
    else:
        train_csv = "./train.csv"
    train_dir = "./train"
    test_dir = "./test"
    ss_dir = "./sample_submission.csv"
else:
    if IS_CORRECTED_TRAIN:
        train_csv = "../input/feedback-prize-corrected-train-csv/corrected_train.csv"
    else:
        train_csv = "../input/feedback-prize-2021/train.csv"
        
    train_directory = "../input/feedback-prize-2021/train"
    test_directory = "../input/feedback-prize-2021/test"
    ss_dir = "../input/feedback-prize-2021/sample_submission.csv"
    

In [ ]:
if IS_CORRECTED_TRAIN:
    print("CAUTION: USING CORRECTED TRAN FILE FROM @nboard")
    
df = pd.read_csv("../input/feedback-prize-corrected-train-csv/corrected_train.csv")
df_ss = pd.read_csv("../input/feedback-prize-2021/sample_submission.csv")


In [ ]:
df["discourse_start"] = df["new_start"]
df["discourse_end"] = df["new_end"]
df["predictionstring"] = df["new_predictionstring"]

In [ ]:
def read_train_file(currid = "423A1CA112E2", curr_dir = train_directory):
    with open(os.path.join(curr_dir, "{}.txt".format(currid)), "r") as f:
        filetext = f.read()
        
    return filetext


In [ ]:
d_labels = df["discourse_type"].unique()
discourse_type_num_labels = {i:x for x,i in enumerate(df["discourse_type_num"].unique())}

In [ ]:
def add_file_features(train, 
                      is_train = True):
    '''
        Dont call directly. Use get_feature_engineered instead
    '''    
    df = train.copy()
    if is_train == False:
        direc = test_directory
    else:
        direc = train_directory
        
    ret = []
    for i in tqdm(df["id"].unique()):
        txt = read_train_file(i, direc)
        filelen = len(txt)
        row = {"id" : i}
        row["non_alpha"] = len(re.findall( "[^a-zA-Z\d\s:]"
                                  , txt))
        row["alpha"] = len(txt) - row["non_alpha"]
        row["numbers"] = len(re.findall("[\d]", txt))
        row["filelength"] = filelen
        row["commas"] = len(re.findall("[,]", txt))
        row["splits"] = len(re.findall("[\n?.,\r!]", txt))
        row["tokens"] = len(txt.split())

        row["first_line_pos"] = txt.find("\n") / filelen
        row["first_ques_pos"] = txt.find("?") / filelen
        row["first_exclaim_pos"] = txt.find("!") / filelen
        row["first_period_pos"] = txt.find(".") / filelen
        row["first_comma_pos"] = txt.find(",") / filelen

        row["last_line_pos"] = txt.rfind("\n") / filelen
        row["last_ques_pos"] = txt.rfind("?") / filelen
        row["last_exclaim_pos"] = txt.rfind("!") / filelen
        row["last_period_pos"] = txt.rfind(".") / filelen
        row["last_comma_pos"] = txt.rfind(",") / filelen

        # for d in d_labels:
        #     row["num_{}".format(d)] = train_pivot.loc[train_pivot["id"] == i, d].values[0]


        ret.append( row )
    return pd.DataFrame(ret)

def merge_num_discourse(train):
    '''
        Dont call directly. Use get_feature_engineered instead.
        This can merge the Number of Discourse types against a feature engineered dataframe.
    '''
    df = train.copy()
    
    train_pivot = df[["id", "discourse_type_num"]].pivot_table(index = ["id"],
                           columns = ["discourse_type_num"],
                           aggfunc = len).reset_index()
    train_pivot = train_pivot.fillna(0)
    
    return train_pivot #df.merge( train_pivot, on = ["id"] )

def merge_file_and_discourse_properties(train, is_train = True):
    df = train.copy()
    ret = []
    if is_train:
        for i in tqdm(df.itertuples(), total = len(df)):
            ret.append( {"id" : getattr(i, "id"),
                         "discourse_type_num" : getattr(i, "discourse_type_num"),
                         "discourse_start" : int(getattr(i, "discourse_start")),
                         "discourse_end" : int(getattr(i, "discourse_end")),
                         "token_start" : int(getattr(i, "predictionstring").split()[0]),
                         "token_end" : int(getattr(i, "predictionstring").split()[-1]),
                        })
    else:
        for i in tqdm(df["id"].unique()):
            for d in discourse_type_num_labels:
                ret.append( {"id" : i,
                             "discourse_type_num" : d})

    return pd.DataFrame(ret)

def get_feature_engineered(train, is_train = True):
    df = train.copy()
    

    
    file_properties = add_file_features(train, is_train = is_train)
    
    discourse_properties = merge_file_and_discourse_properties(df, 
                                                               is_train = is_train)
    
    train = discourse_properties.merge( file_properties,
                              on = ["id"])
    if is_train:
        train["discourse_start"] = train["discourse_start"] / train["filelength"]
        train["discourse_end"] = train["discourse_end"] / train["filelength"]
        train["token_start"] = train["token_start"] / train["tokens"]
        train["token_end"] = train["token_end"] / train["tokens"]
    
    train["discourse_type_num"] = train["discourse_type_num"].map(discourse_type_num_labels)
    return train


def get_r_squared(preds, y_true):
    u = np.sum(( (y_true  - preds) ** 2))

    v = np.sum(  (y_true - np.mean( y_true )) ** 2 )
    score = 1 - u/v
    print(score)
    return score

# Get Features from Text

* Get and merge 2 kinds of features. 
    1. Textual features and splits and positions for training discourse_start and discourse_end predictor
    2. File features against the probability of having a Lead1 or Lead2 present in the text

In [ ]:
# Step 1 : Get the pivot
train_pivot = merge_num_discourse(df)

train = get_feature_engineered(df)

test = get_feature_engineered(df_ss, is_train = False)


# Model 2 Train Adjustments

    * For predicting if a type exists or not, use only file details as inputs
    * Remove the discourse_type_num feature from predictors in this case



In [ ]:
train_pivot = train.merge(train_pivot, on = ["id"])

train_pivot = train_pivot.drop(["discourse_type_num"], axis = 1)

train_pivot = train_pivot.drop_duplicates(subset = ["id"])

# Train/Test Split

* Clean splits based on file ids


In [ ]:
train_size = 0.7

train_inds = np.random.choice( train["id"].unique(),
                              int(train_size * len( train["id"].unique())),
                             replace = False)

X_train = train[train["id"].isin(train_inds)].copy()
X_test = train[~train["id"].isin(train_inds)].copy()

# Targets / Predictors

WE train 2 types of models.

    * First will predict the discourse_start and discourse_end for a category based on the tabular data
    * Second will predict if the said category is actually present in the dataset

Indeed there is no significant information given to the models so the performance cannot be the same as models which use textual & positional information for predictions.


In [ ]:
TARGETS = [x for x in train.columns if (x.find("start") > -1) or (x.find("end") > -1)]
PREDICTORS = [x for x in train.columns if train[x].dtype != "object" and x not in TARGETS]

TARGETS2 = [x for x,i in discourse_type_num_labels.items()]
PREDICTORS2 = [x for x in PREDICTORS if x != "discourse_type_num"]

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

def train_model_and_predict(df_train, targets, predictors, target_df, df_test = None):
    predictions = []
    for ind, i in enumerate(tqdm( targets )):
        print("Training a model for {}".format(i))
        gb = GradientBoostingRegressor(random_state = 91,
                              n_estimators = 500, 
                              n_iter_no_change = 20)
        gb.fit( df_train[predictors], df_train[targets[ind]] )
        if df_test is not None:
            score = gb.score(df_test[predictors],
                            df_test[targets[ind]])
            print("Model R-squared score for {} is {}".format(
                        i, score))
        predictions.append( gb.predict( target_df[predictors] ) )
    
    return predictions

In [ ]:
predictions_01 = train_model_and_predict(X_train, TARGETS, PREDICTORS, test, df_test = X_test)

In [ ]:
predictions_02 = train_model_and_predict(train_pivot, TARGETS2, PREDICTORS2, test, df_test = None)

# Append Predictions to Submission

* Append the predictions to our processed submission dataframe



In [ ]:
for ind, i in enumerate(tqdm(TARGETS)):
    test[TARGETS[ind]] = predictions_01[ind]
for ind, i in enumerate(tqdm(TARGETS2)):
    test[TARGETS2[ind]] = predictions_02[ind] >= 0.5

# Explore Submission DataFrame Now


In [ ]:
test.head(3)

# Discourse to Predictionstring

* Almost same code as from organizers. (Just adds a fail-safe for zero len outputs)


In [ ]:
def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if len(output) == 0:
        return -91
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    
    return output

# Append Predictions

* We add a prediction from Model 1 to final submission **only** if model 2 thinks that this category should be present in the final output

In [ ]:
rev_discourse_type_num_labels = {i:x for x,i in discourse_type_num_labels.items()}

ret = []
for i in tqdm(test["id"].unique()):
    txt = read_train_file(i, test_directory)
    num_chars = len(txt)
    
    for j in test[test["id"] == i].iterrows():
        if j[1][rev_discourse_type_num_labels[ j[1]["discourse_type_num"] ]] == True: 
            # Only process if this entry is true to be processed. Else skip adding this row.
            disc_start = j[1]["discourse_start"]
            disc_end = j[1]["discourse_end"]
            # From relative position to char positions
            disc_start = np.floor(disc_start * num_chars)
            disc_end = np.ceil(disc_end * num_chars)
            
            disc_start = 0 if disc_start < 0 else disc_start
            disc_end = num_chars if disc_end > num_chars else disc_end
            if disc_start > disc_end:
                t = disc_start
                disc_start = disc_end
                disc_end = t
            
            predictionstring = calc_word_indices(txt,
                                                               int(disc_start),
                                                               int(disc_end))
            if predictionstring == -91:
                continue #Invalid calc_position output
                
            predictionstring = " ".join([str(x) for x in predictionstring])
            ret.append( {"id" : i,
                         "predictionstring" : predictionstring,
                         "discourse_type_num" : j[1]["discourse_type_num"]
                        })

# Discourse_type_num to Discourse_type Mapping

* ALl our predictions are discourse type num category. For final submission we should convert it into discourse type only


In [ ]:
disc_mapping = df[["discourse_type_num", "discourse_type"]].drop_duplicates(subset = ["discourse_type_num"]).set_index("discourse_type_num").to_dict()["discourse_type"]


In [ ]:
ret = pd.DataFrame(ret)

# Now map discourse_type_num onto discourse_type


In [ ]:
ret["discourse_type_num"] = ret["discourse_type_num"].map(rev_discourse_type_num_labels)

ret["discourse_type_num"] = ret["discourse_type_num"].map(disc_mapping)


# Prepare Submission


In [ ]:
ret = ret.rename( columns = {"discourse_type_num" : "class"})

ret[["id", "class", "predictionstring"]].to_csv("submission.csv", index = False)

In [ ]:
ret

# That's All Folks